In [ ]:
# Parameters
endpoint_name = None  # injected by Airflow/Papermill
# sns_topic_arn = None 


In [1]:
import boto3

# Config 
# endpoint_name = "xgb-heart-risk-endpoint-20250510T070130Z"  # Update to match your deployed one
alarm_name = f"{endpoint_name}-LatencyAlarm"
# sns_topic_arn = None  # add your SNS ARN for alert emails

# Create CloudWatch alarm 
cloudwatch = boto3.client("cloudwatch")

cloudwatch.put_metric_alarm(
    AlarmName=alarm_name,
    MetricName="InvocationLatency",
    Namespace="AWS/SageMaker",
    Statistic="Maximum",
    Dimensions=[{"Name": "EndpointName", "Value": endpoint_name}],
    Period=300,  # 5 min window
    EvaluationPeriods=1,
    Threshold=1000.0,  # milliseconds
    ComparisonOperator="GreaterThanThreshold",
    AlarmDescription="Alarm if SageMaker endpoint latency exceeds 1 second",
    ActionsEnabled=False if not sns_topic_arn else True,
    AlarmActions=[sns_topic_arn] if sns_topic_arn else [],
)

print(f"CloudWatch latency alarm created: {alarm_name}")

CloudWatch latency alarm created: xgb-heart-risk-endpoint-20250510T070130Z-LatencyAlarm


In [ ]:
import json
import os

# Store alarm metadata for Airflow XCom pull
metadata = {
    "alarm_name": alarm_name
}

os.makedirs("/opt/airflow/out", exist_ok=True)
with open("/opt/airflow/out/notebook_output_metadata.json", "w") as f:
    json.dump(metadata, f)
